### 서울지역 스타벅스 매장정보 크롤링 
- 과제1 : 메인화면 페이지 화면캡쳐 및 저장
- 과제2 : 서울지역 스타벅스 매장정보 크롤링
- 과제3 : 서울인구 정보를 가져온 후 스타벅스지표를 나타낸다. (스벅비율 = alpha x 100 / 스타벅스매장수/인구수)
- 과제4 : 스벅비율 및 스벅매장수를 서울시 지도에 시각화
- 과제5 : 스벅비율이 반영된 파일을 저장

In [1]:
# ! pip install selenium

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.display import Image

import re

In [3]:
%pwd

'C:\\Users\\dxodn\\Desktop\\Github\\Dummy\\Python\\scraping'

In [4]:
driver_path = "../../driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

In [5]:
# 홈페이지 접속
url_page = 'https://www.istarbucks.co.kr/'
driver.get(url_page)

In [6]:
# 과제1: 스크린샷
capture_img = '../images/starbucks_main_screenshot.png'
driver.save_screenshot(capture_img)
Image(capture_img)

TypeError: a bytes-like object is required, not 'str'

TypeError: a bytes-like object is required, not 'str'

<IPython.core.display.Image object>

In [ ]:
# 메뉴창 켜기
driver.find_element_by_css_selector('#wrap > div.tablet_gnb_wrap > nav > ul > li.tablet_gnb04 > a').click()

In [ ]:
# store
driver.find_element_by_css_selector('#wrap > div.mob_gnb_wrap > div > nav > ul:nth-child(4) > li.mob_gnb_ttl2 > a > span').click()

In [ ]:
# 매장 찾기
driver.find_element_by_css_selector('#wrap > div.mob_gnb_wrap > div > nav > ul:nth-child(4) > li:nth-child(3) > a > span').click()

In [ ]:
driver.find_element_by_css_selector('#wrap > div.mob_gnb_wrap > div > nav > ul:nth-child(4) > li:nth-child(3) > ul > li:nth-child(2) > a').click()

In [ ]:
driver.find_element_by_css_selector('#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a').click()

In [ ]:
driver.find_element_by_css_selector('#mCSB_2_container > ul > li:nth-child(1) > a').click()

In [ ]:
current_url = driver.current_url
current_url

In [ ]:
star = requests.get(current_url).text
star = BeautifulSoup(star, 'lxml')
star.title # 제대로 실행했는지 사이트 확인

# 안 되니까 구글링으로 다른 방법으로 찾아보자

In [ ]:
# 이러면 다 찾아올 수 잇다, 이제 매장, 구, 주소 전화번호를 찾자.
all_data = BeautifulSoup(driver.page_source, 'lxml')
all_data.title

In [ ]:
# 이름
qq = [i.text.strip() for i in all_data.select('li strong')]; qq[:10]

In [ ]:
# 주소
all_data.select('p.result_details')

In [ ]:
# 499번째 매장까지 다 들어가 있다, 일단 각각 text로 바꿔주자.
all_data_find = all_data.find_all('li', 'quickResultLstCon'); all_data_find

In [ ]:
all_data_text = [i.text for i in all_data_find]; all_data_text

In [ ]:
# 제목, 구, 주소, 전화번호를 각각 뽑아보자.(구는 주소에서 분리하면 될듯)
name = all_data.select('.quickResultLstCon > strong')
name = [i.text[:-2] for i in name]

add_call = all_data.select('.quickResultLstCon .result_details')
add_call = [str(i) for i in add_call]
add = []
call = []
gu = []
for i in add_call:
    add.append(i[i.find('서울') : i.find('<br/>')])
    call.append(i[i.find('<br/>') + 5: i.find('</p>')])

# 구만 따로 뽑자

for i in add:
    gu.append(i.split(' ')[1])

In [ ]:
# 데이터프레임으로 저장
import pandas as pd

seoul_starbucks_series = {'구' : gu,
                                        '스타벅스 매장' : name,
                                         '전화번호' : call,
                                         '주소': add}
seoul_starbucks = pd.DataFrame(seoul_starbucks_series); seoul_starbucks

In [ ]:
population = pd.read_csv('data/seoul_data/seoul_population.csv', sep=',', encoding='euc-kr')
population.head()

In [ ]:
population.set_index('구', inplace = True); population

In [ ]:
스타벅스 = seoul_starbucks['구'].value_counts()

In [ ]:
스타벅스.name = '스타벅스'; 스타벅스

In [ ]:
population = pd.concat([population, 스타벅스], axis = 1, sort = True)
population.head()

In [ ]:
population['스벅비율'] = (population['스타벅스'] * 100000) / population['인구수']
population['스벅비율']

In [ ]:
population.sort_values(by = '스타벅스', ascending = False).head()

In [ ]:
#! pip install folium

In [ ]:
# 스타벅스 수
import json
import folium
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

geo_path = 'data/seoul_data/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

star = pd.DataFrame({'gu':population.index, 'counts':population['스타벅스']})

map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_str,
              data=star,
              columns=['gu', 'counts'],
              fill_color='PuRd', #PuRd, YlGnBu
              key_on='feature.id')

In [ ]:
map

In [ ]:
# 인구대비 스타벅스 비율



star = pd.DataFrame({'gu':population.index, 'counts':population['스벅비율']})

map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_str,
              data=star,
              columns=['gu', 'counts'],
              fill_color='YlGnBu',
              key_on='feature.id')

In [ ]:
map